Let’s start by specifying: 1. The S3 bucket and prefix to use for the training and model data. This should be within the same region as the Notebook Instance, training, and hosting. 1. The IAM role ARN used to grant access to your data.

In [1]:
import os
import boto3
import re
import sagemaker

role = sagemaker.get_execution_role()
region = sagemaker.Session().boto_region_name

bucket = sagemaker.Session().default_bucket()
prefix = "sagemaker/DEMO-xgboost-dist-script"

## Fetch the dataset

The following methods split the data into train/test/validation datasets and upload files to S3.

In [2]:
import io
import boto3
import random

random.seed(42)


def data_split(
    FILE_DATA,
    DATA_DIR,
    FILE_TRAIN_BASE,
    FILE_TRAIN_1,
    FILE_VALIDATION,
    FILE_TEST,
    PERCENT_TRAIN_0,
    PERCENT_TRAIN_1,
    PERCENT_VALIDATION,
    PERCENT_TEST,
):
    data = [l for l in open(FILE_DATA, "r")]
    train_file_0 = open(DATA_DIR + "/" + FILE_TRAIN_0, "w")
    train_file_1 = open(DATA_DIR + "/" + FILE_TRAIN_1, "w")
    valid_file = open(DATA_DIR + "/" + FILE_VALIDATION, "w")
    tests_file = open(DATA_DIR + "/" + FILE_TEST, "w")

    num_of_data = len(data)
    num_train_0 = int((PERCENT_TRAIN_0 / 100.0) * num_of_data)
    num_train_1 = int((PERCENT_TRAIN_1 / 100.0) * num_of_data)
    num_valid = int((PERCENT_VALIDATION / 100.0) * num_of_data)
    num_tests = int((PERCENT_TEST / 100.0) * num_of_data)

    data_fractions = [num_train_0, num_train_1, num_valid, num_tests]
    split_data = [[], [], [], []]

    rand_data_ind = 0

    for split_ind, fraction in enumerate(data_fractions):
        for i in range(fraction):
            rand_data_ind = random.randint(0, len(data) - 1)
            split_data[split_ind].append(data[rand_data_ind])
            data.pop(rand_data_ind)

    for l in split_data[0]:
        train_file_0.write(l)

    for l in split_data[1]:
        train_file_1.write(l)

    for l in split_data[2]:
        valid_file.write(l)

    for l in split_data[3]:
        tests_file.write(l)

    train_file_0.close()
    train_file_1.close()
    valid_file.close()
    tests_file.close()


def write_to_s3(fobj, bucket, key):
    return (
        boto3.Session(region_name=region)
        .resource("s3")
        .Bucket(bucket)
        .Object(key)
        .upload_fileobj(fobj)
    )


def upload_to_s3(bucket, channel, filename):
    fobj = open(filename, "rb")
    key = prefix + "/" + channel
    url = "s3://{}/{}/{}".format(bucket, key, filename)
    print("Writing to {}".format(url))
    write_to_s3(fobj, bucket, key)

## Ingest data

Next, we read the dataset from the existing repository into memory, for preprocessing prior to training. This processing could be done in situ by SageMaker Processing, Amazon Athena, Apache Spark in Amazon EMR, Amazon Redshift, etc., assuming the dataset is present in the appropriate location. Then, the next step would be to transfer the data to S3 for use in training. For small datasets, such as this one, reading into memory isn’t onerous, though it would be for larger datasets.

In [3]:
s3 = boto3.client("s3")

# Load the dataset
FILE_DATA = "abalone"
s3.download_file(
    "sagemaker-sample-files", f"datasets/tabular/uci_abalone/abalone.libsvm", FILE_DATA
)

In [4]:
# Split the downloaded data into train/test/validation files
FILE_TRAIN_0 = "abalone.train_0"
FILE_TRAIN_1 = "abalone.train_1"
FILE_VALIDATION = "abalone.validation"
FILE_TEST = "abalone.test"
PERCENT_TRAIN_0 = 35
PERCENT_TRAIN_1 = 35
PERCENT_VALIDATION = 15
PERCENT_TEST = 15

DATA_DIR = "data"

if not os.path.exists(DATA_DIR):
    os.mkdir(DATA_DIR)

data_split(
    FILE_DATA,
    DATA_DIR,
    FILE_TRAIN_0,
    FILE_TRAIN_1,
    FILE_VALIDATION,
    FILE_TEST,
    PERCENT_TRAIN_0,
    PERCENT_TRAIN_1,
    PERCENT_VALIDATION,
    PERCENT_TEST,
)

In [5]:
# Upload the files to the S3 bucket
upload_to_s3(bucket, "train/train_0.libsvm", DATA_DIR + "/" + FILE_TRAIN_0)
upload_to_s3(bucket, "train/train_1.libsvm", DATA_DIR + "/" + FILE_TRAIN_1)
upload_to_s3(bucket, "validation/validation.libsvm", DATA_DIR + "/" + FILE_VALIDATION)
upload_to_s3(bucket, "test/test.libsvm", DATA_DIR + "/" + FILE_TEST)

Writing to s3://sagemaker-us-east-1-103721820087/sagemaker/DEMO-xgboost-dist-script/train/train_0.libsvm/data/abalone.train_0
Writing to s3://sagemaker-us-east-1-103721820087/sagemaker/DEMO-xgboost-dist-script/train/train_1.libsvm/data/abalone.train_1
Writing to s3://sagemaker-us-east-1-103721820087/sagemaker/DEMO-xgboost-dist-script/validation/validation.libsvm/data/abalone.validation
Writing to s3://sagemaker-us-east-1-103721820087/sagemaker/DEMO-xgboost-dist-script/test/test.libsvm/data/abalone.test


## Create an XGBoost training script

SageMaker can now run an XGBoost script using the XGBoost estimator. When run on SageMaker, a number of helpful environment variables are available to access properties of the training environment, such as:

- SM_MODEL_DIR: A string representing the path to the directory to write model artifacts to. Any artifacts saved in this folder are uploaded to S3 for model hosting after the training job completes.

- SM_OUTPUT_DIR: A string representing the filesystem path to write output artifacts to. Output artifacts may include checkpoints, graphs, and other files to save, not including model artifacts. These artifacts are compressed and uploaded to S3 to the same S3 prefix as the model artifacts.

When two input channels, train and validation, are used in the call to the XGBoost estimator’s fit() method, the following environment variables are set, following the format SM_CHANNEL_[channel_name]:

- SM_CHANNEL_TRAIN: A string representing the path to the directory containing data in the ‘train’ channel.

- SM_CHANNEL_VALIDATION: Same as above, but for the ‘validation’ channel.

A typical training script loads data from the input channels, configures training with hyperparameters, trains a model, and saves a model to the model_dir so that it can be hosted later. Hyperparameters are passed to your script as arguments and can be retrieved with an argparse.ArgumentParser instance. For example, the script that we run in this notebook is provided as the accompanying file (abalone.py) and also shown below:

In [6]:
!pygmentize abalone.py

import argparse
import json
import logging
import os
import pandas as pd
import pickle as pkl

from sagemaker_containers import entry_point
from sagemaker_xgboost_container.data_utils import get_dmatrix
from sagemaker_xgboost_container import distributed

import xgboost as xgb


def _xgb_train(params, dtrain, evals, num_boost_round, model_dir, is_master):
    """Run xgb train on arguments given with rabit initialized.

    This is our rabit execution function.

    :param args_dict: Argument dictionary used to run xgb.train().
    :param is_master: True if current node is master host in distributed training,
                        or is running single node training job.
                        Note that rabit_run includes this argument.
    """
    booster = xgb.train(params=params,
                        dtrain=dtrain,
                        evals=evals,
                        num_boost_round=num_boost_round)

    if is_master:
        model_location = model_dir + '/xgboost-model'

## Train the XGBoost model

After setting training parameters, we kick off training, and poll for status until training is complete.

To run our training script on SageMaker, we construct a sagemaker.xgboost.estimator.XGBoost estimator, which accepts several constructor arguments:

- entry_point: The path to the Python script that SageMaker runs for training and prediction.

- role: Role ARN

- train_instance_type (optional): The type of SageMaker instances for training.

- sagemaker_session (optional): The session used to train on SageMaker.

- hyperparameters (optional): A dictionary passed to the train function as hyperparameters.


In [7]:
hyperparams = {
    "max_depth": "5",
    "eta": "0.2",
    "gamma": "4",
    "min_child_weight": "6",
    "subsample": "0.7",
    "objective": "reg:squarederror",
    "num_round": "50",
    "verbosity": "2",
}

instance_type = "ml.m5.2xlarge"
output_path = "s3://{}/{}/{}/output".format(bucket, prefix, "abalone-dist-xgb")
content_type = "libsvm"

In [8]:
# Open Source distributed script mode
from sagemaker.session import Session
from sagemaker.inputs import TrainingInput
from sagemaker.xgboost.estimator import XGBoost

session = Session()
script_path = "abalone.py"

xgb_script_mode_estimator = XGBoost(
    entry_point=script_path,
    framework_version="1.5-1",  # Note: framework_version is mandatory
    hyperparameters=hyperparams,
    role=role,
    instance_count=2,
    instance_type=instance_type,
    output_path=output_path,
)

train_input = TrainingInput(
    "s3://{}/{}/{}/".format(bucket, prefix, "train"), content_type=content_type
)
validation_input = TrainingInput(
    "s3://{}/{}/{}/".format(bucket, prefix, "validation"), content_type=content_type
)

## Train an XGBoost Estimator on the Abalone data

Training is as simple as calling fit() on the Estimator. This starts a SageMaker training job that downloads the data, invokes the entry point code (in the provided script file), and saves any model artifacts that the script creates.

In [9]:
xgb_script_mode_estimator.fit({"train": train_input, "validation": validation_input})

2022-11-26 10:49:09 Starting - Starting the training job...ProfilerReport-1669459749: InProgress
...
2022-11-26 10:49:52 Starting - Preparing the instances for training...
2022-11-26 10:50:32 Downloading - Downloading input data...
2022-11-26 10:50:52 Training - Downloading the training image...
2022-11-26 10:51:33 Training - Training image download completed. Training in progress.[2022-11-26 10:51:24.448 ip-10-2-210-198.ec2.internal:7 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2022-11-26:10:51:24:INFO] Imported framework sagemaker_xgboost_container.training
[2022-11-26:10:51:24:INFO] No GPUs detected (normal if no gpus installed)
[2022-11-26:10:51:24:INFO] Invoking user training script.
[2022-11-26:10:51:24:INFO] Module abalone does not provide a setup.py. 
Generating setup.py
[2022-11-26:10:51:24:INFO] Generating setup.cfg
[2022-11-26:10:51:24:INFO] Generating MANIFEST.in
[2022-11-26:10:51:24:INFO] Installing module with the following command:
/miniconda3/bin/python3 -m p

## Deploy the XGBoost model

After training, we use the estimator to create an Amazon SageMaker endpoint – a hosted and managed prediction service to perform inference.

You can also optionally specify other functions to customize the deserialization of the input request (input_fn()), serialization of the predictions (output_fn()), and how predictions are made (predict_fn()). The defaults work for our current use-case so we don’t need to define them.

In [10]:
predictor = xgb_script_mode_estimator.deploy(
    initial_instance_count=1, instance_type="ml.m5.2xlarge"
)

-----!

In [11]:
test_file = DATA_DIR + "/" + FILE_TEST
with open(test_file, "r") as f:
    payload = f.read()

In [13]:
FILE_TEST

'abalone.test'

In [14]:
runtime_client = session.sagemaker_runtime_client
response = runtime_client.invoke_endpoint(
    EndpointName=predictor.endpoint_name, ContentType="text/libsvm", Body=payload
)
result = response["Body"].read().decode("ascii")
print("Predicted values are {}.".format(result))

Predicted values are [10.623578071594238, 7.341342926025391, 13.998539924621582, 7.619755744934082, 11.44434928894043, 12.548134803771973, 11.751919746398926, 19.64179039001465, 9.819439888000488, 8.777353286743164, 9.324228286743164, 8.244194984436035, 12.216837882995605, 8.815969467163086, 13.555924415588379, 10.151961326599121, 11.198446273803711, 10.439631462097168, 9.022027015686035, 10.92051887512207, 6.6454877853393555, 11.439027786254883, 8.975362777709961, 8.689529418945312, 13.65711498260498, 6.642266750335693, 10.187984466552734, 12.229193687438965, 10.684317588806152, 11.267091751098633, 11.386499404907227, 9.72307014465332, 7.123825550079346, 12.945900917053223, 10.218866348266602, 20.112186431884766, 11.871651649475098, 7.491597652435303, 10.606607437133789, 11.973468780517578, 7.996725082397461, 16.567672729492188, 10.705711364746094, 9.892657279968262, 6.224245548248291, 6.50828218460083, 10.298068046569824, 9.435389518737793, 10.863693237304688, 12.21094036102295, 13.1

## Cleanup

When you’re done with this exercise, please run the cell below to delete the hosted endpoint and avoid any additional charges.

In [15]:
predictor.delete_model()
predictor.delete_endpoint()